# Exhaustive Search

This notebook serves to visualize the search space of the `NASbench` problem.
It is a simple brute force search over all possible architectures.
The search space is quite large, making it impossible to map each solution to its corresponding performance, due to memory constraints.
What we can do however, is create a loop in which we still evaluate all possible architectures, and only store the number of times we encounter the performance value.
This way we can still visualize the distribution of the performance values.

Imports

In [ ]:
import os
from itertools import product
import pandas as pd
import matplotlib.pyplot as plt
from nasbench.api import NASBench
from main import init_ipynb, nas_ioh
from utils import get_directories

DIRS = get_directories(os.path.join(os.path.abspath(''), 'exhaustive_search.ipynb'))

In [ ]:
version = 'full'
assert version in ['full', 'only108']

Creating the NASbench object

In [ ]:
NB = NASBench(f'../data/nasbench_{version}.tfrecord')

Defining dummy variables for interop with `main.py`.
They do not have any real impact on what is being done here, as we never call the `main()` function.

In [ ]:
args = dict(optimizer='GA', population_size=100, mu_=40, lambda_=60, budget=5000, recombination='kp', selection='rw', mutation='uniform', xp=1, mut_r=None, mut_b=None, run_id=None, verbose=0, seed=42, repetitions=20, log=False)

We pass the NASbench object to `main.py`

In [ ]:
init_ipynb(NB, args)

Define possibilities for both parts of the bitstring

In [ ]:
mat_bitstrings = [b for b in product([0, 1], repeat=21) if sum(b) <= 9]
print(f'Number of bitstring possibilities for matrix part: {len(mat_bitstrings)}')

ops_bitstrings = [b for b in product([0, 1, 2], repeat=5)]
print(f'Number of bitstring possibilities for operations part: {len(ops_bitstrings)}')

For every matrix bitstring, we create a list of all possible final bitstrings and evaluate them.
The total number of function calls to `nas_ioh()` will be `695,860 * 243 = 196,093,980`.
This might take a while... (4 hours on MacBook Air M1 for 108, 6 hours for full)

In [ ]:
scores = {}
for i, mat_bs in enumerate(mat_bitstrings):
    print(f'\r{i/len(mat_bitstrings)*100:.2f}%', end='')
    for bs in [mat_bs + ops_bs for ops_bs in ops_bitstrings]: scores[score] = 1 if not (score:=nas_ioh(bs)) in scores else scores[score] + 1

In [ ]:
# sort scores by score (descending)
scores = {k: v for k, v in sorted(scores.items(), key=lambda item: item[0], reverse=True)}
# convert to pandas dataframe
scores_df = pd.DataFrame.from_dict(scores, orient='index', columns=['count'])
# save to csv
scores_df.to_csv(DIRS['csv'] + f'scores_dist_{version}.csv')

In [ ]:
scores_df = pd.read_csv(DIRS['csv'] + f'scores_dist_{version}.csv', index_col=0)

We can now plot the distribution of all of the performance values.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(scores_df.index.values, scores_df['count'].values, s=5, alpha=0.5)
ax.set_yscale('log')
ax.set_xlabel('score')
ax.set_ylabel('occurrences')
ax.set_title(f'Distribution of NASbench scores ({version})')
fig.tight_layout()
fig.savefig(DIRS['plots'] + f'scores_dist_{version}.png')